Ahmad Soni - 668564
Nasiem Ayob - 1825850

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "pcm" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "jw300-baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag


# This will save it to a folder in our gdrive instead!
!mkdir -p "en_pcm/$src-$tgt-$tag"
os.environ["experiment_path"] = "en_pcm/%s-%s-%s" % (source_language, target_language, tag)

In [ ]:
!echo $gdrive_path

In [ ]:

# Install opus-tools
! pip install opustools-pkg

In [ ]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-pcm.xml.gz not found. The following files are available for downloading:

        ./JW300_latest_xml_en.zip already exists
        ./JW300_latest_xml_pcm.zip already exists
 289 KB https://object.pouta.csc.fi/OPUS-JW300/v1c/xml/en-pcm.xml.gz

 289 KB Total size
./JW300_latest_xml_en-pcm.xml.gz ... 100% of 289 KB
gzip: JW300_latest_xml_en-pcm.xml already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [ ]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2021-10-08 13:30:35--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en.2’

test.en-any.en.2    100%[===================>] 271.28K  --.-KB/s    in 0.03s   

2021-10-08 13:30:36 (8.12 MB/s) - ‘test.en-any.en.2’ saved [277791/277791]

--2021-10-08 13:30:36--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-pcm.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response

In [ ]:

# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [ ]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 1674/26546 lines since contained in test set.


,source_sentence,target_sentence
0,"WEEK OF JUNE 27 , 2016 – JULY 3 , 2016","JUNE 27 , 2016 – JULY 3 , 2016 WEEK"
1,3 Settle Differences in a Spirit of Love,3 Make Una Dey Use Love Settle Quarrel
2,"Because of our inherited imperfection , we are...",Because of the sin wey all of us carry from be...


###Pre-processing and export


In [ ]:

# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np
from os import cpu_count
from functools import partial
from multiprocessing import Pool


# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

In [ ]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.bpe.en <==
Russell re@@ coun@@ ted that the “ au@@ di@@ ences were very atten@@ tive ” to the sub@@ ject “ The O@@ a@@ th@@ -@@ B@@ ound Promi@@ se ” about Abraham ’ s faith and the future blessings for mankind .
But I thought , ‘ If Jehovah th@@ in@@ ks I can do it , I will try . ’
His class was discu@@ ssing different religi@@ ons , and Lu@@ ca no@@ tic@@ ed that the t@@ ext@@ book conta@@ ined some wrong i@@ de@@ as about Jehovah ’ s Witnesses .
For instance , we appreciate the spiritual food we receive in the form of books , broch@@ u@@ res , magaz@@ ines , and other literature .
For example , he wrote to the congregation in Colo@@ ssa@@ e about false Christians who were tr@@ ying to gain God ’ s favor through works of the Law rather than through faith in Christ .
A C@@ A@@ VA@@ L@@ R@@ Y OF AN@@ GE@@ LI@@ C F@@ OR@@ C@@ ES
The mee@@ k are those who hu@@ mb@@ ly accept Jehovah ’ s teaching and guidance ; the righteous are those who love doing what is right in the eyes of 

### Joey-NMT

In [ ]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .
# Install Pytorch with GPU support v1.7.1.
! pip install torch==1.9.0 -f https://download.pytorch.org/whl/torch_stable.html

fatal: destination path 'joeynmt' already exists and is not an empty directory.
Processing /content/joeynmt
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Using cached torch-1.9.1-cp37-cp37m-manylinux1_x86_64.whl (831.4 MB)
  Using cached torch-1.9.0-cp37-cp37m-manylinux1_x86_64.whl (831.4 MB)
  Created wheel for joeynmt: filename=joeynmt-1.3-py3-none-any.whl size=86029 sha256=7594e2269316d6278d3474c70c8e3261f868bc7669c4088d5b89087a2adaa129
  Stored in directory: /tmp/pip-ephem-wheel-cache-nfcen18v/wheels/0a/f4/bf/6c9d3b8efbfece6cd209f865be37382b02e7c3584df2e28ca4
Successfully built joeynmt
  Attempting uninstall: torch
    Found existing in

### Preproc

In [ ]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data",target_language + source_language ) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$tgt$src/train.bpe.$src joeynmt/data/$tgt$src/train.bpe.$tgt --output_path joeynmt/data/$tgt$src/vocab.txt

# Some output
! echo "BPE Swahili Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$tgt$src/vocab.txt  # Herman

bpe.codes.4000	dev.pcm       test.en-any.en	test.sw        train.pcm
dev.bpe.en	test.bpe.en   test.en-any.en.1	train.bpe.en   vocab.txt
dev.bpe.pcm	test.bpe.pcm  test.en-any.en.2	train.bpe.pcm
dev.en		test.en       test.pcm		train.en
bpe.codes.4000	dev.pcm       test.en-any.en	test.sw        train.pcm
dev.bpe.en	test.bpe.en   test.en-any.en.1	train.bpe.en
dev.bpe.pcm	test.bpe.pcm  test.en-any.en.2	train.bpe.pcm
dev.en		test.en       test.pcm		train.en
BPE Swahili Sentences
( b ) Wetin you go get for mind sey you no go do ?
book . “ Na wetin I learn for Proverbs 27 : ​ 11 , Matthew 26 : ​ 52 , and John 13 : ​ 35 , give me mind wey mek I no join army .
And even when the wahala come , na wetin dey for this Bible verse dem , still help me fit bear the wahala . ” ​ — An@@ dri@@ y , wey come from U@@ k@@ ra@@ ine .
“ Na wetin dey Isaiah 2 : 4 help me mek I no follow fight war , even when dem want mek I join .
I just dey think for my mind how trouble no go dey the new world .
Combined BPE Voc

In [ ]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.pcm       test.en-any.en	test.sw        train.pcm
dev.bpe.en	test.bpe.en   test.en-any.en.1	train.bpe.en
dev.bpe.pcm	test.bpe.pcm  test.en-any.en.2	train.bpe.pcm
dev.en		test.en       test.pcm		train.en


### Joey Config

In [ ]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["experiment_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{experiment_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 200                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 5

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, experiment_path=os.environ["experiment_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

## Training

In [ ]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_reverse_$tgt$src.yaml

2021-10-08 13:46:04,827 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-10-08 13:46:04,938 - INFO - joeynmt.data - Loading training data...
2021-10-08 13:46:05,284 - INFO - joeynmt.data - Building vocabulary...
2021-10-08 13:46:05,578 - INFO - joeynmt.data - Loading dev data...
2021-10-08 13:46:05,591 - INFO - joeynmt.data - Loading test data...
2021-10-08 13:46:05,617 - INFO - joeynmt.data - Data loaded.
2021-10-08 13:46:05,618 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-10-08 13:46:05,876 - INFO - joeynmt.model - Enc-dec model built.
2021-10-08 13:46:12,329 - INFO - joeynmt.training - Total params: 12099072
2021-10-08 13:46:12,332 - WARNING - joeynmt.training - `keep_last_ckpts` option is outdated. Please use `keep_best_ckpts`, instead.
2021-10-08 13:46:12,333 - INFO - joeynmt.helpers - cfg.name                           : pcmen_reverse_transformer
2021-10-08 13:46:12,334 - INFO - joeynmt.helpers - cfg.data.src                       : pcm
2021-10-08

In [ ]:
!mkdir -p "$experiment_path/models/${src}${tgt}_transformer/"

In [ ]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp -r joeynmt/models/${tgt}${src}_reverse_transformer/* "drive/MyDrive/Models/${src}${tgt}_reverse_transformer/"

In [ ]:
# Output our validation accuracy
! cat "drive/MyDrive/Models/enpcm_reverse_transformer/validations.txt"

Steps: 1000	Loss: 98606.76562	PPL: 58.91971	bleu: 1.90166	LR: 0.00030000	*


In [ ]:
# Output our validation accuracy
! cat "drive/MyDrive/Models/enpcm_reverse_transformer/config.yaml"


name: "pcmen_reverse_transformer"

data:
    src: "pcm"
    trg: "en"
    train: "data/pcmen/train.bpe"
    dev:   "data/pcmen/dev.bpe"
    test:  "data/pcmen/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/pcmen/vocab.txt"
    trg_vocab: "data/pcmen/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "/content/drive/My Drive/masakhane/sw-en-baseline/models/pcmen_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000   

In [ ]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "drive/MyDrive/Models/enpcm_reverse_transformer/config.yaml"

Traceback (most recent call last):
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/content/joeynmt/joeynmt/__main__.py", line 48, in <module>
    main()
  File "/content/joeynmt/joeynmt/__main__.py", line 38, in main
    output_path=args.output_path, save_attention=args.save_attention)
  File "/content/joeynmt/joeynmt/prediction.py", line 279, in test
    cfg = load_config(cfg_file)
  File "/content/joeynmt/joeynmt/helpers.py", line 180, in load_config
    with open(path, 'r', encoding="utf-8") as ymlfile:
FileNotFoundError: [Errno 2] No such file or directory: 'drive/MyDrive/Models/enpcm_reverse_transformer/config.yaml'
